# Be very cautious when running this!!

Unsupervised clustering will probably work better, which can be found at `../colab/3.find-relevant-faces-colab.ipynb`

# Here we use the [MELD dataset](https://affective-meld.github.io/) to extract videos and faces.

Download [this small_dataset json](https://raw.githubusercontent.com/cltl/ma-communicative-robots/master/multimodal/small_dataset.json), as well

In [11]:
import json
import csv
import cv2
import numpy as np
from tqdm.notebook import tqdm
import shutil
import os
import random

RAW_TRAIN_VIDEOS_PATH = '/home/tk/datasets/MELD/MELD.Raw/train/train_splits'
TRAIN_SENT_EMO_PATH = '/home/tk/repos/MELD/data/MELD/train_sent_emo.csv'
SMALL_DATASET_PATH = '/home/tk/repos/cltl-face-all/examples/smaller-datasets-jsons/datasets.json'
SAVE_DIR = 'DEBUG'

with open(SMALL_DATASET_PATH, 'r') as stream:
    small_dataset = json.load(stream)

small_dataset = sorted(small_dataset['small']['train'])

with open(TRAIN_SENT_EMO_PATH) as f:
    reader = csv.reader(f)
    data = list(reader)


dia_utts = []
for d in data:
    name = d[2]
    did = d[5]
    uid = d[6]
    vidfile = f"dia{did}_utt{uid}.mp4"

    dia = f"dia{did}"

    if dia in str(small_dataset):
        dia_utts.append((vidfile, name))
    else:
        continue

random.shuffle(dia_utts)
dia_utts = dia_utts[:50]

names_mentioned = [name for du, name in dia_utts]
unique_names = sorted(list(set(names_mentioned)))
important_names = []

for un in unique_names:
    num_occurences = names_mentioned.count(un)
#     if num_occurences > 5:
    print(un, num_occurences)
    important_names.append(un)
print()

print(f"The number of videos BEFORE removing not important name videos: {len(dia_utts)}")

for du_name in list(dia_utts):
    du, name = du_name
    if name not in important_names:
        dia_utts.remove(du_name)

dia_utts.sort(key=lambda x:x[1])

print(f"The number of videos AFTER removing not important name videos: {len(dia_utts)}")

for name in important_names:
    # shutil.rmtree(os.path.join(save_dir, name), ignore_errors=True)
    os.makedirs(os.path.join(SAVE_DIR, name), exist_ok=True)

Ben 2
Chandler 5
Charlie 2
Danny 2
Joey 8
Mischa 2
Monica 6
Phoebe 11
Professore Clerk 1
Rachel 3
Ross 8

The number of videos BEFORE removing not important name videos: 50
The number of videos AFTER removing not important name videos: 50


In [12]:
import av
from cltl_face_all.agegender import AgeGender
from cltl_face_all.arcface import ArcFace
from cltl_face_all.arcface import calc_angle_distance
from cltl_face_all.face_alignment import FaceDetection
import uuid


ag = AgeGender(device='cpu')
af = ArcFace(device='cpu')
fd = FaceDetection(device='cpu', face_detector='blazeface')


batch_size = 64
face_threshold = 0.9


for du, name in tqdm(dia_utts):
    full_video_path = os.path.join(RAW_TRAIN_VIDEOS_PATH, du)

    container = av.open(full_video_path)

    batch = []
    print(full_video_path)
    for idx, frame in enumerate(container.decode(video=0)):
        numpy_RGB = np.array(frame.to_image())
        batch.append(numpy_RGB)

        if len(batch) == batch_size:
            print(len(batch))

            batch = np.stack(batch, axis=0)
            print(batch.shape)
            bboxes = fd.detect_faces(batch)
            print(len(bboxes))
            landmarks = fd.detect_landmarks(batch, bboxes)
            faces = fd.crop_and_align(batch, bboxes, landmarks)
            print(len(faces))
            faces = np.concatenate(faces, axis=0)
            print(faces.shape)
            embeddings = af.predict(faces)
            print(embeddings.shape)

            assert len(faces) == len(embeddings)

            for bb, fa, em in zip(bboxes, faces, embeddings):
                if len(bb) == 0:
                    continue
                prob = bb[0][4]
                if prob > face_threshold:
                    unique_name = str(uuid.uuid4())
                    cv2.imwrite(os.path.join(SAVE_DIR, name, unique_name) + '.jpg', cv2.cvtColor(fa, cv2.COLOR_RGB2BGR))
                    with open(os.path.join(SAVE_DIR, name, unique_name) + '.npy', 'wb') as stream:
                        np.save(stream, em)


            batch = []
    if len(batch) != 0:
        print(len(batch))

        batch = np.stack(batch, axis=0)
        print(batch.shape)
        bboxes = fd.detect_faces(batch)
        print(len(bboxes))
        landmarks = fd.detect_landmarks(batch, bboxes)
        faces = fd.crop_and_align(batch, bboxes, landmarks)
        print(len(faces))
        faces = np.concatenate(faces, axis=0)
        print(faces.shape)
        embeddings = af.predict(faces)
        print(embeddings.shape)

        assert len(faces) == len(embeddings)

        for bb, fa, em in zip(bboxes, faces, embeddings):
            if len(bb) == 0:
                continue

            prob = bb[0][4]
            if prob > face_threshold:
                unique_name = str(uuid.uuid4())
                cv2.imwrite(os.path.join(SAVE_DIR, name, unique_name) + '.jpg', cv2.cvtColor(fa, cv2.COLOR_RGB2BGR))
                unique_name = str(uuid.uuid4())
                with open(os.path.join(SAVE_DIR, name, unique_name) + '.npy', 'wb') as stream:
                    np.save(stream, em)

        batch = []

    print()

[*] load ckpt from /home/tk/.virtualenvs/dev-python-3.7/lib/python3.7/site-packages/cltl_face_all/arcface/./pretrained_models/arc_res50/e_8_b_40000.ckpt


/home/tk/datasets/MELD/MELD.Raw/train/train_splits/dia1029_utt13.mp4
46
(46, 720, 1280, 3)
46
46
(46, 112, 112, 3)
(46, 512)

/home/tk/datasets/MELD/MELD.Raw/train/train_splits/dia1029_utt4.mp4
14
(14, 720, 1280, 3)
14
14
(28, 112, 112, 3)
(28, 512)

/home/tk/datasets/MELD/MELD.Raw/train/train_splits/dia12_utt8.mp4
64
(64, 720, 1280, 3)
64
64
(64, 112, 112, 3)
(64, 512)
8
(8, 720, 1280, 3)
8
8
(8, 112, 112, 3)
(8, 512)

/home/tk/datasets/MELD/MELD.Raw/train/train_splits/dia111_utt17.mp4
64
(64, 720, 1280, 3)
64
64
(99, 112, 112, 3)
(99, 512)
64
(64, 720, 1280, 3)
64
64
(124, 112, 112, 3)
(124, 512)
64
(64, 720, 1280, 3)
64
64
(78, 112, 112, 3)
(78, 512)
64
(64, 720, 1280, 3)
64
64
(39, 112, 112, 3)
(39, 512)
10
(10, 720, 1280, 3)
10
10
(2, 112, 112, 3)
(2, 512)

/home/tk/datasets/MELD/MELD.Raw/train/train_splits/dia1007_utt0.mp4
64
(64, 720, 1280, 3)
64
64
(63, 112, 112, 3)
(63, 512)
2
(2, 720, 1280, 3)
2
2
(2, 112, 112, 3)
(2, 512)

/home/tk/datasets/MELD/MELD.Raw/train/train_splits/d

## Now go through the images and delete the wrong images.

## Let's clean the data

In [13]:
from glob import glob
import os

important_names = ['Chandler', 'Joey', 'Monica', 'Phoebe', 'Rachel', 'Ross']
for name in important_names:
    npys = glob(os.path.join(SAVE_DIR, name, "*.npy"))

    for npy in list(npys):
        jpg = npy.replace('.npy', '.jpg')
        if not os.path.isfile(jpg):
            os.remove(npy)

In [14]:
from cltl_face_all.arcface import calc_angle_distance

for name in important_names:
    print(name)
    npys = glob(os.path.join(SAVE_DIR, name, "*.npy"))
    embs = []

    for npy in npys:
        with open(npy, 'rb') as stream:
            embs.append(np.load(stream))
    embs = np.stack(embs)
    print(embs.shape)

    dists = calc_angle_distance(embs, embs)
    print(dists.round(2))
    print()

    indexes = dists.mean(axis=1).argsort()[:len(dists.mean(axis=1)) // 2]

    print(indexes)
    embs = embs[indexes]
    print(len(embs))


    emb_mean = embs.mean(axis=0)
    emb_final = emb_mean / np.linalg.norm(emb_mean)
    print(emb_final.shape)

    with open(os.path.join(SAVE_DIR, name) + '.npy', 'wb') as stream:
        np.save(stream, emb_final)


Chandler
(141, 512)
[[0.   1.66 1.05 ... 0.78 1.03 0.63]
 [1.66 0.   1.64 ... 1.75 1.73 1.71]
 [1.05 1.64 0.   ... 1.15 1.11 1.07]
 ...
 [0.78 1.75 1.15 ... 0.   1.14 0.48]
 [1.03 1.73 1.11 ... 1.14 0.   1.03]
 [0.63 1.71 1.07 ... 0.48 1.03 0.  ]]

[ 13 110  17  59  49  57  56 101  29 117 131  45  31  34   9  15  23  83
  58  32 111  18  10   3  96  38 105  52  78  43  55 107  50  91   6  41
  36  61  27  80 140 113 119   0 130  30  40  53 139 133  88 115  68  82
  42 122 136 132   7  60  81 114  74 120 108  16 137 118  46  37]
70
(512,)
Joey
(315, 512)
[[0.   0.87 1.31 ... 0.84 1.34 0.82]
 [0.87 0.   1.23 ... 0.62 0.94 0.93]
 [1.31 1.23 0.   ... 1.2  1.39 1.14]
 ...
 [0.84 0.62 1.2  ... 0.   1.09 0.95]
 [1.34 0.94 1.39 ... 1.09 0.   1.29]
 [0.82 0.93 1.14 ... 0.95 1.29 0.  ]]

[297   4 139 211 262  55 118 226 179 209 284 248  53 110  56  83  48  24
 294 210 124  34  94 234 126 205 163   6  78 252  29 149 246 251 212 184
 183 100 129 291  84  70 314  28 103  45 166 198 177 247 231 237 